In [1]:
import logging
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd



In [2]:
# — 로깅 설정 —
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')

In [3]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

service = Service(ChromeDriverManager().install())

# 크롬드라이버 실행
driver = webdriver.Chrome(service = service)

2025-07-17 19:19:54,199 [INFO] ====== WebDriver manager ======
2025-07-17 19:19:55,259 [INFO] Get LATEST chromedriver version for google-chrome
2025-07-17 19:19:55,307 [INFO] Get LATEST chromedriver version for google-chrome
2025-07-17 19:19:55,336 [INFO] Driver [C:\Users\for18\.wdm\drivers\chromedriver\win64\138.0.7204.157\chromedriver-win32/chromedriver.exe] found in cache


In [4]:
# 기사 스크랩하는 과정을 함수로 만들어주기
def scrap_news_title(driver): 

    # 뉴스 기사 제목 스크래핑
    # 제목에 해당하는 요소 선택
    headline_spans = driver.find_elements(By.CSS_SELECTOR, "span.sds-comps-text-type-headline1") 
    # 리스트에 저장
    article_titles = [span.text.strip() for span in headline_spans]

    return article_titles

In [5]:

def scrap_news_titles_simple(url, css_selector, max_scroll=5, wait_time=10):
    try:
        logging.info(f"접속: {url}")
        driver.get(url)

        last_height = driver.execute_script("return document.body.scrollHeight")
        logging.info(f"초기 페이지 높이: {last_height}")

        for i in range(1, max_scroll + 1):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            try:
                WebDriverWait(driver, wait_time).until(
                    lambda d: d.execute_script("return document.body.scrollHeight") > last_height
                )
                new_height = driver.execute_script("return document.body.scrollHeight")
                logging.info(f"  스크롤 {i}: 높이 {last_height} → {new_height}")
                last_height = new_height
            except Exception:
                logging.info(f"  스크롤 {i}: 더 이상 로드할 콘텐츠 없음 (대기 {wait_time}s 초과)")
                break

        # 페이지 소스 파싱 대신, driver에서 직접 헤드라인 추출
        titles = scrap_news_title(driver)
        logging.info(f"크롤링된 뉴스 헤드라인 개수: {len(titles)}")
        return titles

    except Exception as e:
        logging.error(f"크롤링 중 오류 발생: {e}")
        return []

    finally:
        driver.quit()
        logging.info("드라이버 종료")

In [6]:
#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://www.naver.com/')

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

In [7]:
# 검색어 창을 찾아 search 변수에 저장 (By.XPATH 방식)
search_box = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input')


In [8]:
search_box.click()
search_box.clear()    # 혹시 이전 값이 있으면 지우고
search_box.send_keys("물냉면", Keys.RETURN)
time.sleep(1)

In [9]:
# (기존 검색 클릭 후) 결과를 로딩할 시간 확보
time.sleep(1)
news_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[8]')
news_button.click()

In [10]:
titles = []
if __name__ == "__main__":
    url = driver.current_url
    for idx, title in enumerate(scrap_news_titles_simple(url, css_selector=None), 1):
        print(f"{idx}. {title}")
        titles.append(title)

2025-07-17 19:20:03,869 [INFO] 접속: https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=%EB%AC%BC%EB%83%89%EB%A9%B4
2025-07-17 19:20:04,159 [INFO] 초기 페이지 높이: 2860
2025-07-17 19:20:05,467 [INFO]   스크롤 1: 높이 2860 → 4781
2025-07-17 19:20:05,991 [INFO]   스크롤 2: 높이 4781 → 6704
2025-07-17 19:20:06,514 [INFO]   스크롤 3: 높이 6704 → 8552
2025-07-17 19:20:07,042 [INFO]   스크롤 4: 높이 8552 → 10282
2025-07-17 19:20:07,569 [INFO]   스크롤 5: 높이 10282 → 12012
2025-07-17 19:20:08,207 [INFO] 크롤링된 뉴스 헤드라인 개수: 60
2025-07-17 19:20:10,616 [INFO] 드라이버 종료


1. 오뚜기, '고기고명 평양물냉면' 육수 별도 출시
2. ‘고객 입맛 되찾아라’ 본아이에프, 여름맞이 프로모션 전개
3. [오늘 뭐 먹지] 빙그레·오뚜기·네스프레소 외
4. [유통 브리핑] 대상·오뚜기·농심
5. CU, 마이노멀과 저당 간편식 선봬…곤약 물냉면·비빔밥·떡볶이까지
6. 물냉면 vs 비빔냉면… 혈당 걱정인 사람, ‘이것’ 선택을
7. [유통가 레이더] "물 들어왔다 노 저어라"... 농심, 美 LA 갤럭시 홈구장...
8. 풀무원, 골프장 여름 시즌 메뉴
9. [식품워치] 식품업계, 신제품 출시·팝업·이벤트·콜라보·이벤트·캠페...
10. [유통24시] 한화갤러리아 '벤슨' 아이스크림, 스타벅스 판매 外
11. [조승연 PD의 맛썰] 평양냉면 순례 2
12. ‘우래옥 대기 130팀’… MZ의 ‘힙푸드’ 평양냉면 열전
13. CU, 마이노멀과 저당 간편식 출시…곤약 물냉면·비빔밥·떡볶이까지
14. "간편식 냉면 나트륨 주의해야…하루 기준치 초과 제품도"
15. '복면가왕' '물냉면'은 오정태 "현빈 되려면 수술비 3천만 원...죽을 수...
16. [오늘의 언박싱] 오뚜기 ‘평양냉면 육수’·하이트진로 ‘일품진로 25년...
17. [AT 유통]GS25·CU·이마트24
18. '물냉면'→오정태, '버스 안내원'→현영이었다… "둘이 붙으면 재밌었을...
19. [신상NEWS] CU ‘마이노멀 저당 전주식 비빔밥’·청정원 ‘짜지 않은 리...
20. 후루룩 들이켜는 여름의 맛 [떴다! 기자평가단]
21. "CJ 동치미물냉면908g 3490원" 요기요 '요마트 여름 빅세일페스타'
22. 이른 무더위 '여름면 성수기' 길어진다…CJ·농심·팔도·풀무원·오뚜기...
23. [오늘의 장바구니] CU·HDC아이파크몰·코리아나화장품·해비치 외
24. CJ제일제당, 냉면 4종 리뉴얼…"외식 전문점 맛 구현"
25. 냉면부터 쫄면까지…CJ제일제당, 여름면 4종 리뉴얼 출시
26. "충주 한복판 3500원짜리 해장국·물냉면 아시나요"
27. [조승연 PD의 맛썰] 평양냉

In [11]:
df = pd.DataFrame({
    "headline": titles
})
df.to_excel("news_headlines.xlsx", index=False)
